In [167]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import gensim

min_count = 2
size = 50
window = 4
word2vec_model = Word2Vec(min_count=min_count, size=size, window=window)
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('googlenews.vectors.bin', binary=True)
print('Done..')

Done..


In [236]:
def read_text_file(f):
    df_complete = pd.read_csv(f)
    return df_complete

reviewsDFComplete = read_text_file("finland_hotel_reviews.csv")
#print(reviewsDFComplete['review'])

reviewsDFComplete['review'] = reviewsDFComplete['review'].apply(lambda row: str(row).lower())
reviewsDFComplete['tokens']=reviewsDFComplete.apply(lambda row: word_tokenize(str(row['review'])), axis=1)  # Search for all non-letters

stop = stopwords.words('english')
# Keeping stopwords increased accuracy
# Remove stopwords , numeric and alphanumeric characters
#reviewsDFComplete['tokens'] = reviewsDFComplete['review'].apply(lambda row: [row for row in row.split() if row not in stop and row.isalpha()])
reviewsDFComplete['tokens'] = reviewsDFComplete['tokens'].apply(lambda row: ",".join(row))
reviewsDFComplete['review_length'] = reviewsDFComplete.apply(lambda row: len(row['tokens']), axis=1)
reviewvector = np.zeros(shape=(reviewsDFComplete.shape[0], 300))
for index, row in reviewsDFComplete.iterrows():
    review_word = list(filter(lambda word: word in word2vec_model.vocab, row['tokens'].split(',')))
    #print(review_word)
    review_vector_word=[]
    if len(review_word) > 0:
        for word in review_word:
            review_vector_word.append(word2vec_model[word])  # Vector representation of each word in review
    
    #print(review_vector_word)
    # vector of 300 dims representation of each review
    review_vector = np.array(np.sum(review_vector_word, axis=0) / np.sqrt((np.sum(review_vector_word, axis=0) ** 2).sum()))
    #print(len(review_vector), len(review_vector_word))
    review_vector = review_vector.reshape(1, review_vector.shape[0])
    reviewvector[index]=review_vector
    
for i in range(0, 300):
    reviewsDFComplete["review_vector_" + str(i)] = reviewvector[:, i]
        
print('Done')

Done


In [247]:
reviewsDF = reviewsDFComplete
reviewsDF = reviewsDF.drop('review_title',1)
reviewsDF = reviewsDF.drop('address',1)
reviewsDF = reviewsDF.drop('lat',1)
reviewsDF = reviewsDF.drop('lng',1)
reviewsDF = reviewsDF.drop('rating_date',1)
reviewsDF = reviewsDF.drop('review',1)
reviewsDF = reviewsDF.drop('tokens',1)
reviewsDF = reviewsDF.drop('name',1)
reviewsDF = reviewsDF.drop('reviewer_nationality',1)
reviewsDF = reviewsDF.drop('tags',1)

#ratingsDF = (reviewsDF['rating'] >= 4).astype('int')
ratingsDF = reviewsDF['rating']
print(reviewsDF.head())
print(ratingsDF.head())

   current_price_per_night  average_rating  total_reviews_received  rating  \
0                      123             4.5                  2018.0     5.0   
1                      123             4.5                  2018.0     5.0   
2                      123             4.5                  2018.0     5.0   
3                      123             4.5                  2018.0     3.0   
4                      123             4.5                  2018.0     5.0   

   review_length  review_vector_0  review_vector_1  review_vector_2  \
0            268        -0.023419         0.061145        -0.000526   
1            225        -0.054742         0.049039         0.008964   
2            265         0.002349         0.040305        -0.013221   
3            247         0.053206         0.040323         0.037969   
4            249         0.016664         0.030725         0.024672   

   review_vector_3  review_vector_4        ...          review_vector_290  \
0         0.105235        -

In [248]:
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression    
    
training_data, test_data, train_target, test_target = train_test_split(reviewsDF, ratingsDF, train_size=0.8, test_size=0.2)
model = LogisticRegression()
#print(training_data.head(), train_target.head())
ovr = OneVsRestClassifier(model).fit(training_data, train_target)
dummy = DummyClassifier('most_frequent')
dummy.fit(training_data, train_target)

#predicting & comparing
predictedOvr = ovr.predict(test_data)
predictedDummy = dummy.predict(test_data)

print('Done')

Done


In [249]:
print('Real data')
print(np.asarray(test_target))
print('Done')

Real data
[ 5.  4.  5. ...,  5.  4.  5.]
Done


In [250]:
print('Majority classifier result')
print(predictedDummy)
print('Majority classifier accuracy:', accuracy_score(test_target,predictedDummy))
print('Done')

Majority classifier result
[ 5.  5.  5. ...,  5.  5.  5.]
Majority classifier accuracy: 0.404494382022
Done


In [251]:
print('OVR predictor result')
print(predictedOvr)
print('OVR accuracy:', accuracy_score(test_target,predictedOvr))
print('Done')

OVR predictor result
[ 5.  4.  5. ...,  5.  4.  5.]
OVR accuracy: 0.83595505618
Done


In [252]:
lr = LogisticRegression()

lr.fit(training_data, train_target)
print(accuracy_score(test_target, lr.predict(test_data)))

0.845505617978


In [253]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(training_data, train_target)

print(accuracy_score(test_target, rf.predict(test_data)))

0.826404494382


In [254]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=3, population_size=3, verbosity=2, max_eval_time_mins=2)
tpot.fit(training_data, train_target)
print(tpot.score(test_data, test_target))

Optimization Progress:  50%|█████     | 6/12 [01:06<01:14, 12.41s/pipeline]

Generation 1 - Current best internal CV score: 1.0


Optimization Progress:  75%|███████▌  | 9/12 [02:51<01:20, 26.98s/pipeline]

Generation 2 - Current best internal CV score: 1.0


Generation 3 - Current best internal CV score: 1.0

Best pipeline: GaussianNB(input_matrix)
1.0


In [258]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(training_data, train_target)
print(accuracy_score(test_target, rf.predict(test_data)))

0.826404494382
